# Save to S3 with a SageMaker Processing Job

<div class="alert alert-info"> 💡 <strong> Quick Start </strong>
To save your processed data to S3, select the Run menu above and click <strong>Run all cells</strong>. 
<strong><a style="color: #0397a7 " href="#Job-Status-&-S3-Output-Location">
    <u>View the status of the export job and the output S3 location</u></a>.
</strong>
</div>


This notebook executes your Data Wrangler Flow `Insurance_preprocessed.flow` on the entire dataset using a SageMaker 
Processing Job and will save the processed data to S3.

This notebook saves data from the step `Manage Columns`. To save from a different step, go to Data Wrangler 
to select a new step to export. 

---

## Contents

1. [Inputs and Outputs](#Inputs-and-Outputs)
1. [Run Processing Job](#Run-Processing-Job)
   1. [Job Configurations](#Job-Configurations)
   1. [Create Processing Job](#Create-Processing-Job)
   1. [Job Status & S3 Output Location](#Job-Status-&-S3-Output-Location)
1. [Optional Next Steps](#(Optional)Next-Steps)
    1. [Load Processed Data into Pandas](#(Optional)-Load-Processed-Data-into-Pandas)
    1. [Train a model with SageMaker](#(Optional)Train-a-model-with-SageMaker)
---

# Inputs and Outputs

The below settings configure the inputs and outputs for the flow export.

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

In <b>Input - Source</b> you can configure the data sources that will be used as input by Data Wrangler

1. For S3 sources, configure the source attribute that points to the input S3 prefixes
2. For all other sources, configure attributes like query_string, database in the source's 
<b>DatasetDefinition</b> object.

If you modify the inputs the provided data must have the same schema and format as the data used in the Flow. 
You should also re-execute the cells in this section if you have modified the settings in any data sources.
</div>

In [6]:
import sagemaker
sagemaker.__version__

'2.38.0'

In [2]:
pip install --upgrade sagemaker

     |████████████████████████████████| 409 kB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 989 kB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 371 kB/s  eta 0:00:01
     |████████████████████████████████| 108 kB 77.3 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.39.0-py2.py3-none-any.whl size=572994 sha256=bbf08ce234a6a3c617bdceac8b82aa0842bf7ad9803aaacfbe4aeb5fd0331918
  Stored in directory: /root/.cache/pip/wheels/f7/18/ad/e03d3ecc02b7bbde733308f179b13a29b93c60db223432ae2b
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 0.1.4
    Uninstalling smdebug-rulesconfig-0.1.4:
      Successfully uninstalled smdebug-rulesconfig-0.1.4
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 1.72.1
    Uninstalling sagemaker-1.72.1:
      Successfully uninstalled sagemaker-1.72.1
You should consider upgrading via the '/opt/conda/bin/pyt

In [5]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

data_sources = []

## Input - S3 Source: df_freq.csv

In [4]:
data_sources.append(ProcessingInput(
    source="s3://sagemaker-us-east-1-989682844397/data_wrangler/insurance_dataset/raw_data/df_freq.csv", # You can override this to point to other dataset on S3
    destination="/opt/ml/processing/df_freq.csv",
    input_name="df_freq.csv",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
))

## Input - S3 Source: df_sev.csv

In [5]:
data_sources.append(ProcessingInput(
    source="s3://sagemaker-us-east-1-989682844397/data_wrangler/insurance_dataset/raw_data/df_sev.csv", # You can override this to point to other dataset on S3
    destination="/opt/ml/processing/df_sev.csv",
    input_name="df_sev.csv",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
))

## Output: S3 settings

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

1. <b>bucket</b>: you can configure the S3 bucket where Data Wrangler will save the output. The default bucket from 
the SageMaker notebook session is used. 
2. <b>flow_export_id</b>: A randomly generated export id. The export id must be unique to ensure the results do not 
conflict with other flow exports 
3. <b>s3_ouput_prefix</b>:  you can configure the directory name in your bucket where your data will be saved.
</div>

In [9]:
import time
import uuid
import sagemaker 


# Sagemaker session
sess = sagemaker.Session()


# You can configure this with your own bucket name, e.g.
# bucket = "my-bucket"
bucket = sess.default_bucket()
print(f"Data Wrangler export storage bucket: {bucket}")

# unique flow export ID
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"
print(f"flow_export_name: {flow_export_name}")

Data Wrangler export storage bucket: sagemaker-us-east-1-989682844397
flow_export_name: flow-09-17-40-46-93d28f7e


Below are the inputs required by the SageMaker Python SDK to launch a processing job.

In [13]:
# Output name is auto-generated from the select node's ID + output name from the flow file.
output_name = "95b11c5e-0e03-47c7-bff3-e18d1d0c8ee1.default"

s3_output_prefix = f"export-{flow_export_name}/output"
s3_output_path = f"s3://{bucket}/{s3_output_prefix}"
print(f"Flow S3 export result path: {s3_output_path}")

processing_job_output = ProcessingOutput(
    output_name=output_name,
    source="/opt/ml/processing/output",
    destination=s3_output_path,
    s3_upload_mode="EndOfJob"
)

Flow S3 export result path: s3://sagemaker-us-east-1-989682844397/export-flow-09-17-40-46-93d28f7e/output


## Upload Flow to S3

To use the Data Wrangler as an input to the processing job,  first upload your flow file to Amazon S3.

In [9]:
import os
import json
import boto3

# name of the flow file which should exist in the current notebook working directory
flow_file_name = "Insurance_preprocessed.flow"

# Load .flow file from current notebook working directory 
!echo "Loading flow file from current notebook working directory: $PWD"

with open(flow_file_name) as f:
    flow = json.load(f)

# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(flow_file_name, bucket, f"data_wrangler_flows/{flow_export_name}.flow")

flow_s3_uri = f"s3://{bucket}/data_wrangler_flows/{flow_export_name}.flow"

print(f"Data Wrangler flow {flow_file_name} uploaded to {flow_s3_uri}")

Loading flow file from current notebook working directory: /root/Untitled Folder 2
Data Wrangler flow Insurance_preprocessed.flow uploaded to s3://sagemaker-us-east-1-989682844397/data_wrangler_flows/flow-03-00-16-26-3f79913b.flow


The Data Wrangler Flow is also provided to the Processing Job as an input source which we configure below.

In [10]:
## Input - Flow: Insurance_preprocessed.flow
flow_input = ProcessingInput(
    source=flow_s3_uri,
    destination="/opt/ml/processing/flow",
    input_name="flow",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
)

# Run Processing Job 
## Job Configurations

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

You can configure the following settings for Processing Jobs. If you change any configurations you will 
need to re-execute this and all cells below it by selecting the Run menu above and click 
<b>Run Selected Cells and All Below</b>

1. IAM role for executing the processing job. 
2. A unique name of the processing job. Give a unique name every time you re-execute processing jobs
3. Data Wrangler Container URL.
4. Instance count, instance type and storage volume size in GB.
5. Content type for each output. Data Wrangler supports CSV as default and Parquet.
6. Network Isolation settings
</div>

In [ ]:
# IAM role for executing the processing job.
iam_role = sagemaker.get_execution_role()

# Unique processing job name. Give a unique name every time you re-execute processing jobs
processing_job_name = f"data-wrangler-flow-processing-{flow_export_id}"

# Data Wrangler Container URL.
container_uri = "663277389841.dkr.ecr.us-east-1.amazonaws.com/sagemaker-data-wrangler-container:1.x"

# Processing Job Instance count and instance type.
instance_count = 2
instance_type = "ml.m5.4xlarge"

# Size in GB of the EBS volume to use for storing data during processing
volume_size_in_gb = 30

# Content type for each output. Data Wrangler supports CSV as default and Parquet.
output_content_type = "CSV"

# Network Isolation mode; default is off
enable_network_isolation = False

# Output configuration used as processing job container arguments 
output_config = {
    output_name: {
        "content_type": output_content_type
    }
}

## Create Processing Job

To launch a Processing Job, you will use the SageMaker Python SDK to create a Processor function.

In [12]:
from sagemaker.processing import Processor
from sagemaker.network import NetworkConfig

processor = Processor(
    role=iam_role,
    image_uri=container_uri,
    instance_count=instance_count,
    instance_type=instance_type,
    volume_size_in_gb=volume_size_in_gb,
    network_config=NetworkConfig(enable_network_isolation=enable_network_isolation),
    sagemaker_session=sess
)

# Start Job
processor.run(
    inputs=[flow_input] + data_sources, 
    outputs=[processing_job_output],
    arguments=[f"--output-config '{json.dumps(output_config)}'"],
    wait=False,
    logs=False,
    job_name=processing_job_name
)


Job Name:  data-wrangler-flow-processing-03-00-16-26-3f79913b
Inputs:  [{'InputName': 'flow', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-989682844397/data_wrangler_flows/flow-03-00-16-26-3f79913b.flow', 'LocalPath': '/opt/ml/processing/flow', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'df_freq.csv', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-989682844397/data_wrangler/insurance_dataset/raw_data/df_freq.csv', 'LocalPath': '/opt/ml/processing/df_freq.csv', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'df_sev.csv', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-989682844397/data_wrangler/insurance_dataset/raw_data/df_sev.csv', 'LocalPath': '/opt/ml/processing/df_sev.csv', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}

## Job Status & S3 Output Location

Below you wait for processing job to finish. If it finishes successfully, the raw parameters used by the 
Processing Job will be printed

In [13]:
s3_job_results_path = f"s3://{bucket}/{s3_output_prefix}/{processing_job_name}"
print(f"Job results are saved to S3 path: {s3_job_results_path}")

job_result = sess.wait_for_processing_job(processing_job_name)
job_result

Job results are saved to S3 path: s3://sagemaker-us-east-1-989682844397/export-flow-03-00-16-26-3f79913b/output/data-wrangler-flow-processing-03-00-16-26-3f79913b
....................................................................................................!

{'ProcessingInputs': [{'InputName': 'flow',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-989682844397/data_wrangler_flows/flow-03-00-16-26-3f79913b.flow',
    'LocalPath': '/opt/ml/processing/flow',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'df_freq.csv',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-989682844397/data_wrangler/insurance_dataset/raw_data/df_freq.csv',
    'LocalPath': '/opt/ml/processing/df_freq.csv',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'df_sev.csv',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-989682844397/data_wrangler/insurance_dataset/raw_data/df_sev.csv',
    'LocalPath': '/opt/ml/processing/df_sev.csv',
    'S3DataType': 'S3Prefix',
    'S3InputMod

## (Optional)Next Steps

Now that data is available on S3 you can use other SageMaker components that take S3 URIs as input such as 
SageMaker Training, Built-in Algorithms, etc. Similarly you can load the dataset into a Pandas dataframe 
in this notebook for further inspection and work. The examples below show how to do both of these steps.

By default optional steps do not run automatically, set `run_optional_steps` to True if you want to 
execute optional steps

In [ ]:
run_optional_steps = False

In [ ]:
# This will stop the below cells from executing if "Run All Cells" was used on the notebook.
if not run_optional_steps:
    raise SystemExit("Stop here. Do not automatically execute optional steps.")

### (Optional) Load Processed Data into Pandas

We use the [AWS Data Wrangler library](https://github.com/awslabs/aws-data-wrangler) to load the exported 
dataset into a Pandas dataframe for a preview of first 1000 rows.

In [2]:
!pip install -q awswrangler pandas
import awswrangler as wr

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
ERROR: redshift-connector 2.0.878 has requirement pytz<2021.9,>=2020.1, but you'll have pytz 2019.3 which is incompatible.
ERROR: awswrangler 2.7.0 has requirement pandas<1.3.0,>=1.1.0, but you'll have pandas 1.0.1 which is incompatible.


In [27]:
output_content_type = "CSV"
s3_output_path = 's3://sagemaker-us-east-1-989682844397/export-flow-03-00-16-26-3f79913b/output'

In [28]:
chunksize = 1000

if output_content_type.upper() == "CSV":
    dfs = wr.s3.read_csv(s3_output_path, chunksize=chunksize)
elif output_content_type.upper() == "PARQUET":
    dfs = wr.s3.read_parquet(s3_output_path, chunked=chunksize)
else:
    print(f"Unexpected output content type {output_content_type}") 

df = next(dfs)
df

,VehAge_bin0_3,VehAge_bin3_9,VehAge_bin9_100,DrivAge_bin18_38,DrivAge_bin38_51,DrivAge_bin51_100,IDpol,VehAge,DrivAge,Density,...,Area_D,Area_E,Area_A,Area_B,Area_F,Area,VehPower,VehBrand,VehGas,Region
0,0.0,1.0,0.0,1.0,0.0,0.0,1226.0,7.0,28.0,65.0,...,0.0,0.0,0.0,1.0,0.0,B,7.0,B12,Diesel,R21
1,0.0,1.0,0.0,0.0,0.0,1.0,5244.0,4.0,69.0,7.0,...,0.0,0.0,1.0,0.0,0.0,A,12.0,B12,Diesel,R93
2,0.0,1.0,0.0,0.0,0.0,1.0,21024.0,6.0,81.0,112.0,...,0.0,0.0,0.0,0.0,0.0,C,6.0,B13,Diesel,R24
3,0.0,1.0,0.0,0.0,1.0,0.0,21701.0,8.0,39.0,29.0,...,0.0,0.0,1.0,0.0,0.0,A,4.0,B4,Diesel,R24
4,0.0,1.0,0.0,0.0,0.0,1.0,21940.0,6.0,65.0,16.0,...,0.0,0.0,1.0,0.0,0.0,A,6.0,B2,Diesel,R24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,1.0,0.0,0.0,1.0,71362.0,12.0,68.0,98.0,...,0.0,0.0,0.0,1.0,0.0,B,9.0,B13,Diesel,R31
996,0.0,0.0,1.0,0.0,1.0,0.0,71372.0,11.0,48.0,114.0,...,0.0,0.0,0.0,0.0,0.0,C,5.0,B2,Regular,R31
997,0.0,1.0,0.0,0.0,1.0,0.0,71374.0,8.0,48.0,114.0,...,0.0,0.0,0.0,0.0,0.0,C,6.0,B2,Diesel,R31
998,0.0,1.0,0.0,1.0,0.0,0.0,71387.0,8.0,33.0,1895.0,...,1.0,0.0,0.0,0.0,0.0,D,6.0,B3,Regular,R31


In [29]:
df.columns

Index(['VehAge_bin0_3', 'VehAge_bin3_9', 'VehAge_bin9_100', 'DrivAge_bin18_38',
       'DrivAge_bin38_51', 'DrivAge_bin51_100', 'IDpol', 'VehAge', 'DrivAge',
       'Density', 'ClaimNb', 'Exposure', 'BonusMalus', 'ClaimAmount',
       'VehBrand_B12', 'VehBrand_B1', 'VehBrand_B2', 'VehBrand_B3',
       'VehBrand_B5', 'VehBrand_B6', 'VehBrand_B4', 'VehBrand_B10',
       'VehBrand_B11', 'VehBrand_B13', 'VehBrand_B14', 'VehPower_6',
       'VehPower_7', 'VehPower_5', 'VehPower_4', 'VehPower_8', 'VehPower_10',
       'VehPower_9', 'VehPower_11', 'VehPower_12', 'VehPower_13',
       'VehPower_15', 'VehPower_14', 'VehGas_Regular', 'VehGas_Diesel',
       'Region_R24', 'Region_R82', 'Region_R93', 'Region_R11', 'Region_R53',
       'Region_R52', 'Region_R91', 'Region_R72', 'Region_R31', 'Region_R54',
       'Region_R73', 'Region_R41', 'Region_R25', 'Region_R26', 'Region_R23',
       'Region_R22', 'Region_R83', 'Region_R74', 'Region_R94', 'Region_R21',
       'Region_R42', 'Region_R43', 'Area_C'

#### Rename columns

In [32]:
df.rename(columns = {'VehAge_bin0_3':'VehAge_bin0_0_3_0',
                     'VehAge_bin3_9':'VehAge_bin3_0_9_0',
                    'VehAge_bin9_100':'VehAge_bin9_0_100_0',
                    'DrivAge_bin18_38':'DrivAge_bin18_0_38_0',
                    'DrivAge_bin38_51':'DrivAge_bin38_0_51_0',
                    'DrivAge_bin51_100':'DrivAge_bin51_0_100_0',
                    }, inplace = True)

In [33]:
df

,VehAge_bin0_0_3_0,VehAge_bin3_0_9_0,VehAge_bin9_0_100_0,DrivAge_bin18_0_38_0,DrivAge_bin38_0_51_0,DrivAge_bin51_0_100_0,IDpol,VehAge,DrivAge,Density,...,Area_D,Area_E,Area_A,Area_B,Area_F,Area,VehPower,VehBrand,VehGas,Region
0,0.0,1.0,0.0,1.0,0.0,0.0,1226.0,7.0,28.0,65.0,...,0.0,0.0,0.0,1.0,0.0,B,7.0,B12,Diesel,R21
1,0.0,1.0,0.0,0.0,0.0,1.0,5244.0,4.0,69.0,7.0,...,0.0,0.0,1.0,0.0,0.0,A,12.0,B12,Diesel,R93
2,0.0,1.0,0.0,0.0,0.0,1.0,21024.0,6.0,81.0,112.0,...,0.0,0.0,0.0,0.0,0.0,C,6.0,B13,Diesel,R24
3,0.0,1.0,0.0,0.0,1.0,0.0,21701.0,8.0,39.0,29.0,...,0.0,0.0,1.0,0.0,0.0,A,4.0,B4,Diesel,R24
4,0.0,1.0,0.0,0.0,0.0,1.0,21940.0,6.0,65.0,16.0,...,0.0,0.0,1.0,0.0,0.0,A,6.0,B2,Diesel,R24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,1.0,0.0,0.0,1.0,71362.0,12.0,68.0,98.0,...,0.0,0.0,0.0,1.0,0.0,B,9.0,B13,Diesel,R31
996,0.0,0.0,1.0,0.0,1.0,0.0,71372.0,11.0,48.0,114.0,...,0.0,0.0,0.0,0.0,0.0,C,5.0,B2,Regular,R31
997,0.0,1.0,0.0,0.0,1.0,0.0,71374.0,8.0,48.0,114.0,...,0.0,0.0,0.0,0.0,0.0,C,6.0,B2,Diesel,R31
998,0.0,1.0,0.0,1.0,0.0,0.0,71387.0,8.0,33.0,1895.0,...,1.0,0.0,0.0,0.0,0.0,D,6.0,B3,Regular,R31


In [ ]:
df_transformed.to_csv('dataset_after_FE.csv', index = False)

## (Optional)Train a model with SageMaker
Now that the data has been processed, you may want to train a model using the data. The following shows an 
example of doing so using a popular algorithm - XGBoost. For more information on algorithms available in 
SageMaker, see [Getting Started with SageMaker Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html). 
It is important to note that the following XGBoost objective ['binary', 'regression', 'multiclass'] 
hyperparameters, or content_type may not be suitable for the output data, and will require changes to 
train a proper model. Furthermore, for CSV training, the algorithm assumes that the target 
variable is in the first column. For more information on SageMaker XGBoost, 
see https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html.


### Set Training Data path
We set the training input data path from the output of the Data Wrangler processing job..

In [ ]:
s3_training_input_path = s3_job_results_path
print(f"training input data path: {s3_training_input_path}")

### Configure the algorithm and training job

The Training Job hyperparameters are set. For more information on XGBoost Hyperparameters, 
see https://xgboost.readthedocs.io/en/latest/parameter.html.

In [ ]:
region = boto3.Session().region_name
container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
hyperparameters = {
    "max_depth":"5",
    "objective": "reg:squarederror",
    "num_round": "10",
}
train_content_type = (
    "application/x-parquet" if output_content_type.upper() == "PARQUET"
    else "text/csv"
)
train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_training_input_path,
    content_type=train_content_type,
)

### Start the Training Job

The TrainingJob configurations are set using the SageMaker Python SDK Estimator, and which is fit using 
the training data from the Processing Job that was run earlier.

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    iam_role,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
)
estimator.fit({"train": train_input})

Now that you have a trained model there are a number of different things you can do. 
For more details on training with SageMaker, please see 
https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/using_xgboost.html.